In [1]:
import numpy as np

# from qiskit import *
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import IBMQ, Aer, execute, assemble, transpile
from qiskit.visualization import plot_histogram, plot_bloch_vector
from qiskit.visualization import plot_state_qsphere, plot_state_city, plot_bloch_multivector
from qiskit.visualization import array_to_latex
import qiskit.quantum_info as qi
#from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ionq import IonQProvider
# import gates
from qiskit_ionq import GPIGate, GPI2Gate, MSGate
from qiskit import Aer, execute, assemble

from qiskit_ionq import IonQProvider
from qiskit import Aer, execute, assemble

#import appropriate circuit constructions
from ionq_circuit_constructor import ionq_alice_prepares

In [2]:
def ionq_baseline_measurement_prep(qc, basis):
    if basis == 'X':
        #qc.h(0)
        qc.append(GPI2Gate(1/4), [0])
        qc.append(GPIGate(0), [0])
     
    if basis == 'Y':
        #qc.rx(np.pi/2, 0)
        qc.append(GPI2Gate(0), [0])
        
    return qc
    

In [3]:
def ionq_baseline_circuit(bitval, basis):
    qc = QuantumCircuit(1,1)
    
    qc = ionq_alice_prepares(qc, bitval, basis)
    qc = ionq_baseline_measurement_prep(qc, basis)
    
    qc.measure(0,0)
    
    return qc

In [4]:
def ionq_baseline_experiment(bitval, basis, shots):
    qc = ionq_baseline_circuit(bitval, basis)
    
    provider = IonQProvider("RmK0yNkCDPmoxCH12uQ4U67lpu9kFgik")
    native_simulator = provider.get_backend("ionq_simulator", gateset="native")
    job = native_simulator.run(qc, shots = shots)
    
    probs = job.get_probabilities()
    
    return probs
    

## Simulation Results

In [5]:
# X basis, bitval = 0
ionq_baseline_experiment(0, 'X', 1024)

{'0': 1}

In [6]:
# X basis, bitval = 1
ionq_baseline_experiment(1, 'X', 1024)

{'1': 1}

In [7]:
# Y basis, bitval = 0
ionq_baseline_experiment(0, 'Y', 1024)

{'0': 1}

In [8]:
# Y basis, bitval = 1
ionq_baseline_experiment(1, 'Y', 1024)

{'1': 1}

In [9]:
def ionq_baseline_experiment_QPU(bitval, basis, shots):
    qc = ionq_baseline_circuit(bitval, basis)
    
    provider = IonQProvider("RmK0yNkCDPmoxCH12uQ4U67lpu9kFgik")
    ionq = provider.get_backend("ionq_qpu", gateset="native")
    job = ionq.run(qc, backend = ionq, shots = shots)
    
    # Hardik's code
    #native_qpu = provider.get_backend("ionq_qpu", gateset="native")
    #job = native_qpu.run(qc)
    
    job_id = job.job_id()    
    return qc, job_id

In [18]:
def ionq_baseline_probs_QPU(job_id):
    provider = IonQProvider("RmK0yNkCDPmoxCH12uQ4U67lpu9kFgik")
    ionq = provider.get_backend("ionq_qpu")
    job = ionq.retrieve_job(job_id)
    probs = job.get_probabilities()
    return probs
    

## QPU Results

In [12]:
# x basis, bitvval = 0
qc, job_id = ionq_baseline_experiment_QPU(0, 'X', 1024)
print(job_id)
qc.draw()

/tmp/ipykernel_15355/2916827727.py:6: UserWarning: Option backend is not used by this backend
  job = ionq.run(qc, backend = ionq, shots = shots)


78e63e37-31d6-4bf5-9a2f-a2db6762bc94


┌────────────┐┌────────┐┌────────────┐┌────────┐┌─┐
  q: ┤ Gpi2(0.25) ├┤ Gpi(0) ├┤ Gpi2(0.25) ├┤ Gpi(0) ├┤M├
     └────────────┘└────────┘└────────────┘└────────┘└╥┘
c: 1/═════════════════════════════════════════════════╩═
                                                      0

In [19]:
X0probs = ionq_baseline_probs_QPU(job_id)

In [20]:
print(X0probs)

{'0': 0.990234375, '1': 0.009765625}


In [21]:
# X basis, bitval = 1
qc, job_id = ionq_baseline_experiment_QPU(1, 'X', 1024)
print(job_id)
qc.draw()

/tmp/ipykernel_15355/2916827727.py:6: UserWarning: Option backend is not used by this backend
  job = ionq.run(qc, backend = ionq, shots = shots)


f80835bd-2c37-4309-8d2d-8ba21bd5855c


┌────────┐┌────────────┐┌────────┐┌────────────┐┌────────┐┌─┐
  q: ┤ Gpi(0) ├┤ Gpi2(0.25) ├┤ Gpi(0) ├┤ Gpi2(0.25) ├┤ Gpi(0) ├┤M├
     └────────┘└────────────┘└────────┘└────────────┘└────────┘└╥┘
c: 1/═══════════════════════════════════════════════════════════╩═
                                                                0

In [22]:
X1probs = ionq_baseline_probs_QPU(job_id)
print(X1probs)

{'0': 0.0146484375, '1': 0.9853515625}


In [23]:
# Y basis, bitval = 0
qcY0, job_idY0 = ionq_baseline_experiment_QPU(0, 'Y', 1024)
print(job_idY0)
qcY0.draw()

/tmp/ipykernel_15355/2916827727.py:6: UserWarning: Option backend is not used by this backend
  job = ionq.run(qc, backend = ionq, shots = shots)


3a820579-5ecb-4cf2-88b9-89a01d1a7bcd


┌────────────┐┌────────────┐┌─────────┐┌─┐
  q: ┤ Gpi2(0.25) ├┤ Gpi(0.125) ├┤ Gpi2(0) ├┤M├
     └────────────┘└────────────┘└─────────┘└╥┘
c: 1/════════════════════════════════════════╩═
                                             0

In [24]:
Y0probs = ionq_baseline_probs_QPU(job_idY0)
print(Y0probs)

{'0': 0.994140625, '1': 0.005859375}


In [25]:
# Y basis, bitval = 1
qcY1, job_idY1 = ionq_baseline_experiment_QPU(1, 'Y', 1024)
print(job_idY1)
qcY1.draw()

/tmp/ipykernel_15355/2916827727.py:6: UserWarning: Option backend is not used by this backend
  job = ionq.run(qc, backend = ionq, shots = shots)


92c5c2ed-6a95-4cf1-befb-de2e2a863c51


┌────────────┐┌─────────────┐┌─────────┐┌─┐
  q: ┤ Gpi2(0.25) ├┤ Gpi(-0.125) ├┤ Gpi2(0) ├┤M├
     └────────────┘└─────────────┘└─────────┘└╥┘
c: 1/═════════════════════════════════════════╩═
                                              0

In [26]:
Y1probs = ionq_baseline_probs_QPU(job_idY1)
print(Y1probs)

{'0': 0.005859375, '1': 0.994140625}
